테슬라 주가 예측


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import yfinance as yf

# Get Tesla's stock data from Yahoo Finance
ticker = "TSLA"
df = yf.download(ticker, start="2013-01-01", end="2023-01-01")
df.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
df.rename(columns={'Close': 'close'}, inplace=True)

# Scale the stock data
scaler = MinMaxScaler()
df = scaler.fit_transform(df)

# Split the data into training and test sets
train_data = df[:int(df.shape[0] * 0.8), :]
test_data = df[int(df.shape[0] * 0.8):, :]

# Build the LSTM model
look_back = 10
model = Sequential()
model.add(LSTM(10, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
X_train = np.array([train_data[i:i + look_back, 0] for i in range(train_data.shape[0] - look_back)])
y_train = np.array([train_data[i + look_back, 0] for i in range(train_data.shape[0] - look_back)])
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=0)

# Test the model
X_test = np.array([test_data[i:i + look_back, 0] for i in range(test_data.shape[0] - look_back)])
y_test = np.array([test_data[i + look_back, 0] for i in range(test_data.shape[0] - look_back)])
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
y_pred = model.predict(X_test)

# Inverse transform the predicted data
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))

# Plot the actual and predicted stock prices
plt.plot(test_data[:, 0], label="Actual Stock Price")
plt.plot(y_pred, label="Predicted Stock Price")
plt.legend()
plt.show()


accuracy = model.evaluate(X_test, y_test, verbose=0)
# Print the accuracy of the model
print("Accuracy: {:.2f}%".format(accuracy * 100))
#Get the last look_back data points of the training set as input for prediction
last_look_back_data = df[-look_back:, 0].reshape(1, 1, look_back)

#Make the prediction for 2023
prediction = model.predict(last_look_back_data)

#Inverse transform the predicted data
prediction = scaler.inverse_transform(prediction.reshape(-1, 1))

# Print the predicted stock price for 2023
print("The predicted stock price for 2023 is: {:.2f}".format(prediction[0][0]))

KeyboardInterrupt: 